# TODO Questions
* 7.2, 7.3, 7.4, 7.5
* 8.1(a,b), 8.5, 8.7, 8.11, 8.16(a,b,d)
* 9.1, 9.2, 9.3, 9.7, 9.8

# Question 7.1
Both parts (a) and (b) are complete.

In [21]:
"""Horner's Rule Polynomial Calculation."""

import numpy as np


def horner(degree, coeffs, t):
    """Apply the horner algorithm to a polynomial at t.
    
    Args:
        degree: Degree of the polynomial.
        coeffs: The coefficients of the polynomial.
        t: The value of the independent variable we evaluate the function at.
    """
    b = float(coeffs[degree])

    for i in reversed(range(degree)):
        b = coeffs[i] + b * t

    return b


def evaluate(degree, coeffs, t=None, a=None, b=None, evaluate='function'):
    """
    Wrapper of the horner algorithm to allow the computation of the
    function as is, or its derivative at t, or its definite integral with
    bounds (b, a).
    """
    coeffs = np.array(coeffs)
    
    if evaluate == 'function' and t is not None:
        return horner(degree, coeffs, t)
    
    elif evaluate == 'derivative' and t is not None:
        # Do the derivative on the coefficients.
        for i in range(degree):
            coeffs[i] = float(coeffs[i + 1]) * (i + 1)
        
        return horner(degree - 1, coeffs[:degree], t)
    
    elif evaluate == 'integral' and a is not None and b is not None:
        # Integrate the coefficients.
        new_coeffs = np.zeros(degree + 2)
        
        for i in reversed(range(1, degree + 2)):
            new_coeffs[i] = float(coeffs[i - 1]) / i
        
        at_a = horner(degree + 1, new_coeffs, a)
        at_b = horner(degree + 1, new_coeffs, b)
        return at_a - at_b
    
    else:
        return "You messed up."


# Test case.
print(evaluate(2, np.array([0, 2, 1]), t=2))
print(evaluate(2, np.array([0, 2, 1]), t=2, evaluate='derivative'))
print(evaluate(1, np.array([2, 2]), a=2, b=0, evaluate='integral'))

8.0
6.0
8.0


# Question 7.2

In [47]:
"""Newton Polynomial Interpolation Problem."""

import numpy as np


class NewtonInterpolant(object):
    def __init__(self, data):
        self.data = np.array(data)
        self.n = self.data.shape[0]
        self.A = np.zeros([self.n, self.n])
        self.A[:,0] = np.ones([self.n])

        for i in range(1, self.n):
            for j in range(1, self.n):
                self.A[i, j] = self.A[i, j-1] * (self.data[i, 0]
                                                 - self.data[j-1, 0])

        self.t = np.array(self.data[:, :1])
        self.y = np.array(self.data[:, 1:])
        self.x = np.linalg.solve(self.A, self.y)

    def interpolant(self):
        return np.array(self.x)

    def add(self, datapoint):
        pass

    def evaluate_horner(self, t):
        """Technically not horner..."""
        cumul_t = 1
        result = 0
        for i in range(self.n):
            result += self.x[i] * cumul_t
            cumul_t *= (t - self.t[i])

        return result


evaluator = NewtonInterpolant([[-2,-27],[0,-1], [1,0]])
evaluator.evaluate_horner(1)
# TODO: New interpolant when data point is added.
# TODO: Recursively implement part a, i.e. given a dataset,
#       incrementally add each one.

array([0.])

In [24]:
np.array([[-2,-27],[0,-1], [1,0]]).shape

(3, 2)